# Oligo Search Website

**Here are the functions I used for the search feature in my website.**

In [1]:
import xlrd
oligo_sheet = (xlrd.open_workbook("kh_oligos.xlsx")).sheet_by_index(0)
reference = (open("reference.txt")).read()

print (reference)

GCGGGCGTGGAGATGGCTTCTAACTTTACTCATGGCTTCTAACTTTACTCAGTTCGTTAGTTCGTTAAGCGAGATTCATTCAAGCCATGGCTTCTAACTTTACTCAGTTCGTTGGGAGGCGATGGACGCCAAACACGACGCCGGCGGGCGTGATTCGAGCATGGCTTCTAACTTTACTCAGTTCGTTGGCGGATTCGAGATTCCGGGCGTGCGGGCGTGAAGCGCG


In [2]:
def reverse_complement(text):
    text = text[::-1].upper().replace(' ','')
    reverse_complement_text = text.translate(str.maketrans('ACGT','TGCA'))
    return reverse_complement_text


def oligo_search(reference, oligo_sheet):
    oligo_column = 2
    reference = reference.upper().replace(' ','')
    rc_reference = reverse_complement(reference)
    matching_oligos = []

    for i in range(oligo_sheet.nrows):
        oligo = (oligo_sheet.cell_value(
                    rowx=i,
                    colx=oligo_column)
                    .upper().replace(" ",""))
        if i < oligo_sheet.nrows and oligo != "" and ((oligo in reference) or (oligo in rc_reference)):
            matching_oligos.append(oligo_sheet.cell_value(rowx=i, colx=oligo_column-1))

    if not matching_oligos:
        return("No oligos match the reference.")
    else:
        return(matching_oligos)


oligo_search(reference, oligo_sheet)

['MS2-F-Pbase-cPCR']

# In order to determine the best way to find the reverse complement and search for a matching oligo, I compared several functions.
## Reverse compliment:

**Python's maketrans() method is the quickest and most readable way to obtain the reverse complement out of the five ways I tested. Below, the function is called "reverse_complement_four".**

In [3]:
from timeit import timeit
from Bio.Seq import Seq

oligo = (oligo_sheet.cell_value(
                    rowx=6,
                    colx=2)
                    .upper().replace(" ",""))
print (oligo)

ATGGCTTCTAACTTTACTCAGTTCGTT


In [4]:
def reverse_complement_one(text):
    text = text.upper().replace(' ','')
    dict_basepair = {'G':'C', 'C':'G', 'A':'T', 'T':'A'}
    reverse_complement_string = ''
    for i, nucleotide in enumerate(text):
        basepair = (text[((i+1)*-1)+len(text)])
        if dict_basepair.get(basepair):
            reverse_complement_string += dict_basepair.get(basepair)
    return(reverse_complement_string)
reverse_complement_one(oligo)

'AACGAACTGAGTAAAGTTAGAAGCCAT'

In [5]:
def reverse_complement_two(text):
    text = text.upper().replace(' ','')
    dict_basepair = {'G':'C', 'C':'G', 'A':'T', 'T':'A'}
    reverse = (text[::-1])
    reverse_complement_string = ''
    for i in reverse:
        if dict_basepair.get(i):
            reverse_complement_string += dict_basepair.get(i)
    return(reverse_complement_string)
reverse_complement_two(oligo)

'AACGAACTGAGTAAAGTTAGAAGCCAT'

In [9]:
def reverse_complement_three(text):
    text = text.upper().replace(' ','')
    dict_basepair = {'G':'C', 'C':'G', 'A':'T', 'T':'A'}
    reverse_complement_string = ''
    for nucleotide in reversed(text):
        if dict_basepair.get(nucleotide):
            reverse_complement_string += dict_basepair.get(nucleotide)
    return (reverse_complement_string)
reverse_complement_three(oligo)

'AACGAACTGAGTAAAGTTAGAAGCCAT'

In [10]:
def reverse_complement_four(text):
    text = text[::-1].upper().replace(' ','')
    reverse_complement_text = text.translate(str.maketrans('ACGT','TGCA'))
    return reverse_complement_text
reverse_complement_four(oligo)

'AACGAACTGAGTAAAGTTAGAAGCCAT'

In [11]:
def reverse_complement_biopython(text):
    sequence = Seq(text)
    reverse_complement_sequence = Seq.reverse_complement(sequence)
    return(reverse_complement_sequence)
reverse_complement_biopython(oligo)

Seq('AACGAACTGAGTAAAGTTAGAAGCCAT', Alphabet())

## Timeit:

**I used timeit to compare the speed of each function after 100,000 completions.**

In [12]:
print ("reverse_complement_one:{}".format(timeit(
                                    "reverse_complement_one(text)",
                                    "from __main__ import reverse_complement_one;"
                                    "text='A@GG GTTAG--TGACCAGCT! AG'", number =100000)))
print ("reverse_complement_two:{}".format(timeit(
                                    "reverse_complement_two(text)",
                                    "from __main__ import reverse_complement_two;"
                                    "text = 'A@GG GTTAG--TGACCAGCT! AG'", number =100000)))
print ("reverse_complement_three:{}".format(timeit(
                                    "reverse_complement_three(text)",
                                    "from __main__ import reverse_complement_three;"
                                    "text = 'A@GG GTTAG--TGACCAGCT! AG'", number =100000)))
print ("reverse_complement_four:{}".format(timeit(
                                    "reverse_complement_four(text)",
                                    "from __main__ import reverse_complement_four;"
                                    "text = 'A@GG GTTAG--TGACCAGCT! AG'", number =100000)))
print ("reverse_complement_biopython:{}".format(timeit(
                                    "reverse_complement_biopython(text)",
                                    "from __main__ import reverse_complement_biopython;"
                                    "text = 'A@GG GTTAG--TGACCAGCT! AG'", number =100000)))

reverse_complement_one:2.547016844999689
reverse_complement_two:1.6029819020000105
reverse_complement_three:1.4778021470001477
reverse_complement_four:0.40757576699979836
reverse_complement_biopython:1.5530386730001737


## Oligo Search:

**I chose to use "if pattern in text: do something" because it's quick and best suited for my website. Below, it is called "pattern_search_three".**

In [13]:
def pattern_search_one(text, pattern):
    count = 0
    for i, nucleotide in enumerate(text):
        if text[i:i+len(pattern)] == pattern:
            count += 1
    return (count)
pattern_search_one(reference, oligo)

3

In [14]:
def pattern_search_two(text, pattern):
    return (text.count(pattern))
pattern_search_one(reference, oligo)

3

In [15]:
def pattern_search_three(text, pattern):
    match = []
    if pattern in text:
        match.append(oligo)
    if not match:
        return ("No pattern found in text.")
    else:
        return ("Pattern found in text.")
pattern_search_three(reference, oligo)

'Pattern found in text.'

In [16]:
def pattern_search_four(text, pattern):
    match = text.find(pattern)
    if match > -1:
        return ("Pattern found in text.")
    else:
        return ("Pattern not found in text.")
pattern_search_four(reference, oligo)

'Pattern found in text.'

In [17]:
print ("pattern_search_one:{}".format(timeit(
                                    "pattern_search_one(text, pattern)",
                                    "from __main__ import pattern_search_one;"
                                    "text='TGATTCCGGCGGGCGTGGAGAAGCGAGATTCATTCAAGCCGGGAGGCGTGGCGTGGCGTGGCGTGCGGATTCAAGCCGGCGGG';"
                                    "pattern='TTCCGG' ", number =100000)))

print ("pattern_search_two:{}".format(timeit(
                                    "pattern_search_two(text, pattern)",
                                    "from __main__ import pattern_search_two;"
                                    "text='TGATTCCGGCGGGCGTGGAGAAGCGAGATTCATTCAAGCCGGGAGGCGTGGCGTGGCGTGGCGTGCGGATTCAAGCCGGCGGG';"
                                    "pattern='TTCCGG' ", number =100000)))

print ("pattern_search_three:{}".format(timeit(
                                    "pattern_search_three(text, pattern)",
                                    "from __main__ import pattern_search_three;"
                                    "text='TGATTCCGGCGGGCGTGGAGAAGCGAGATTCATTCAAGCCGGGAGGCGTGGCGTGGCGTGGCGTGCGGATTCAAGCCGGCGGG';"
                                    "pattern='TTCCGG' ", number =100000)))

print ("pattern_search_four:{}".format(timeit(
                                    "pattern_search_four(text, pattern)",
                                    "from __main__ import pattern_search_four;"
                                    "text='TGATTCCGGCGGGCGTGGAGAAGCGAGATTCATTCAAGCCGGGAGGCGTGGCGTGGCGTGGCGTGCGGATTCAAGCCGGCGGG';"
                                    "pattern='TTCCGG' ", number =100000)))

pattern_search_one:4.815226697000071
pattern_search_two:0.09268529500013756
pattern_search_three:0.10271668499990483
pattern_search_four:0.12597850700012714


## Oligo Search Locations:

**In the future I may want to incorporate the location of the oligo within the reference. I found regular expressions to be the best way to do this.**

In [18]:
import re

In [19]:
def pattern_find_one(text, pattern):
    list = []
    for i, nucleotide in enumerate(text):
        if text[i:i+len(pattern)] == pattern:
            list.append(i)
    return (list)
pattern_find_one(reference, oligo)

[31, 86, 160]

In [20]:
def pattern_find_two(text,pattern):
    matches = re.finditer('(?={0})'.format(pattern),text)
    positions = [str(match.start()) for match in matches]
    result = ' '.join(positions)
    return(result)
pattern_find_two(reference, oligo)

'31 86 160'

In [21]:
def pattern_find_three(text,pattern):
    results = []
    regex_pattern = re.compile('(?={0})'.format(pattern))
    matches = regex_pattern.finditer(text)
    for match in matches:
        results.append(match.start())
    return(results)
pattern_find_three(reference, oligo)

[31, 86, 160]

In [22]:
print ("pattern_find_one:{}".format(timeit(
                                    "pattern_find_one(text, pattern)",
                                    "from __main__ import pattern_find_one;"
                                    "text='TGATTCCGGCGGGCGTGGAGAAGCGAGATTCATTCAAGCCGGGAGGCGTGGCGTGGCGTGGCGTGCGGATTCAAGCCGGCGGG';"
                                    "pattern='TTCCGG' ", number =100000)))

print ("pattern_find_two:{}".format(timeit(
                                    "pattern_find_two(text, pattern)",
                                    "from __main__ import pattern_find_two;"
                                    "text='TGATTCCGGCGGGCGTGGAGAAGCGAGATTCATTCAAGCCGGGAGGCGTGGCGTGGCGTGGCGTGCGGATTCAAGCCGGCGGG';"
                                    "pattern='TTCCGG' ", number =100000)))

print ("pattern_find_three:{}".format(timeit(
                                    "pattern_find_three(text, pattern)",
                                    "from __main__ import pattern_find_three;"
                                    "text='TGATTCCGGCGGGCGTGGAGAAGCGAGATTCATTCAAGCCGGGAGGCGTGGCGTGGCGTGGCGTGCGGATTCAAGCCGGCGGG';"
                                    "pattern='TTCCGG' ", number =100000)))

pattern_find_one:5.226146290000088
pattern_find_two:1.4456950449998658
pattern_find_three:1.2177825789999588
